In [1]:
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException

In [2]:
df = pd.read_csv('./data/hotel_5star_agoda.csv')
df.head()

Unnamed: 0                                         hotel_name hotel_star  \
0           0                그랜드 워커힐 서울 (Grand Walkerhill Seoul)        5성급   
1           1            스위스 그랜드 호텔 서울 (Swiss Grand Hotel Seoul)        5성급   
2           2                  메이필드 호텔 서울 (Mayfield Hotel Seoul)        5성급   
3           3  워커힐 더글라스 하우스 (성인 전용) (Walkerhill Douglas House...        5성급   
4           4                        롯데호텔 월드 (Lotte Hotel World)        5성급   

   hotel_rate                                          hotel_url  \
0         8.8  https://www.agoda.com/ko-kr/grand-walkerhill-s...   
1         8.0  https://www.agoda.com/ko-kr/grand-hilton-seoul...   
2         8.6  https://www.agoda.com/ko-kr/mayfield-hotel/hot...   
3         9.1  https://www.agoda.com/ko-kr/walkerhill-douglas...   
4         8.9  https://www.agoda.com/ko-kr/lotte-hotel-world/...   

                         hotel_location  \
0         워커힐로 177, 광진, 서울, 대한민국, 04963   
1    서대문구 연희로 353, 서대문, 서울, 대한민국, 03650   
2          방화대로 94, 강서, 서울, 대한민국, 07506   
3  서울 광진구 워커힐로 177, 광진, 서울, 대한민국, 04963   
4         올림픽로 240 , 송파, 서울, 대한민국, 5554   

                                              review  
0  [['10.0', '호텔 직원들은 매우 친절했습니다. 체크인부터 프론트 데스크에서 ...  
1  [['10.0', '동생이랑 서울여행 와서 2박 3일 동안 호텔에 머문게 젤 기억에...  
2  [['10.0', '서울 어떤 호텔보다 직원분들이 친절합니다. 체크인부터 수영장이용...  
3  [['10.0', '워커힐은 제가 6살때 난생처음 야외 수영장으로 방문했던 곳으로,...  
4  [['10.0', '지난 여름에 이어서 두번째 방문이었습니다. 롯데월드와 가깝다는 ...

In [ ]:
df['review'] = None
df

In [21]:
urls = df['hotel_url'].tolist()
len(urls)

39

In [8]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [121]:
reviews = []

for i in range(9,11):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(5)
    
    # 언어 변경
    try:
        language_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select'))
        # driver.find_element(By.XPATH, '//*[@id="reviewFilterSection"]/div[1]/div[3]/span/i[2]').click() #21번 전까지
        time.sleep(1)
    except ElementClickInterceptedException:
        continue
    
    # "한국어"를 선택
    try:
        language_dropdown.select_by_visible_text("한국어")
        # driver.find_element(By.XPATH, '//*[@id="reviews-language-filter_list"]/ul/li[2]').click() # 21번 전까지
        time.sleep(1)
    except ElementClickInterceptedException:
        continue
    
    while True:
        star_elements = driver.find_elements(By.CLASS_NAME, 'Review-comment-leftScore')
        review_elements = driver.find_elements(By.CLASS_NAME, 'Review-comment-bodyText')

        for star_element, review_element in zip(star_elements, review_elements):
            score = star_element.text
            review = review_element.text
            reviews.append([score, review])

        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
            time.sleep(1)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break
        except NoSuchElementException:
            break
        
    df.at[i,'review'] = reviews
    reviews = []
    
    time.sleep(160)
    continue

In [ ]:
for i in range(len(urls)):
    review_column = df.iloc[i]['review']
    
    if review_column is not None:
        print(f'{i}번째 : ', len(review_column))
    else:
        print(f'{i}번째 : None.')

In [125]:
df.to_csv('./data/hotel_5star_agoda_add.csv',index=False,encoding='utf-8')

=========================================


4성급 호텔


In [2]:
df_4star = pd.read_csv('./data/hotel_4star_agoda.csv')
df_4star.head()

Unnamed: 0                                         hotel_name hotel_star  \
0           0                 인터시티 서울 호텔 (Intercity Seoul Hotel)        4성급   
1           1       코트야드 서울 보타닉파크 (Courtyard Seoul Botanic Park)        4성급   
2           2  나인트리 프리미어 호텔 인사동 (Nine Tree Premier Hotel Insa...        4성급   
3           3                               유니언 호텔 (UNION HOTEL)        4성급   
4           4  포포인츠 바이 쉐라톤 서울 구로 (Four Points by Sheraton Seo...        4성급   

   hotel_rate                                          hotel_url  \
0         7.7  https://www.agoda.com/ko-kr/intercity-seoul-ho...   
1         9.0  https://www.agoda.com/ko-kr/courtyard-seoul-bo...   
2         8.9  https://www.agoda.com/ko-kr/nine-tree-premier-...   
3         9.1  https://www.agoda.com/ko-kr/seoul-yeongdeungpo...   
4         8.4  https://www.agoda.com/ko-kr/best-western-premi...   

                                  hotel_location  \
0            마곡중앙 6로 76-3, 강서, 서울, 대한민국, 157-201   
1               마곡중앙 12로 10, 강서, 서울, 대한민국, 07789   
2          종로구 인사동길 49, 명동, 인사동, 서울, 대한민국, 03145   
3  unionhotel.sy@gmail.com, 영등포, 서울, 대한민국, 07205   
4          구로구 구로동 1128-1, 구로, 서울, 대한민국, 152-050   

                                              review  
0  [['10.0', '위치, 청결, 직원들의 상냥한 서비스, 시설 등등 모든것이 좋았...  
1  [['10.0', '서울식물원에 갈 겸 친구들과 예약을 했어요. 체크인 할 때부터 ...  
2  [['10.0', '어디로든 이동하기 좋은 위치이고 주변에 관광지를 모두 걸어서 이...  
3  [['10.0', '나는 호텔에서 멋진 시간을 보냈습니다. 그 지역은 붐비는 혼잡한...  
4  [['10.0', '호캉스하려고 집근처 가까운데 보던중 예약했습니다. 성인2명 조식...

In [3]:
df_4star['review'] = None
df_4star.tail()

Unnamed: 0                                         hotel_name hotel_star  \
104         104                             호텔 엘리에나 (HOTEL ELIENA)        4성급   
105         105                     G3 호텔 충무로 (G3 Hotel Chungmuro)        4성급   
106         106  솔라리아 니시테츠 호텔 서울 명동 (Solaria Nishitetsu Hotel S...        4성급   
107         107                     G2 호텔 명동 (G2 Hotel Myeongdong)        4성급   
108         108                     호텔 28 명동 (Hotel 28 Myeongdong)        4성급   

     hotel_rate                                          hotel_url  \
104         8.7  https://www.agoda.com/ko-kr/h33814979/hotel/se...   
105         8.9  https://www.agoda.com/ko-kr/g3-hotel-chungmuro...   
106         9.0  https://www.agoda.com/ko-kr/solaria-nishitetsu...   
107         8.8  https://www.agoda.com/ko-kr/g2-hotel-myeongdon...   
108         9.2  https://www.agoda.com/ko-kr/hotel-28-myeongdon...   

                                        hotel_location review  
104                서울 강남구 논현로 645, 강남, 서울, 대한민국, 06116   None  
105              서울특별시 중구 충무로 7, 명동, 서울, 대한민국, 100-271   None  
106              명동8길 27, 7-22F, 명동, 서울, 대한민국, 100-809   None  
107                        수표로 24, 명동, 서울, 대한민국, 04555   None  
108  13 Myeongdong 7 gil, Jung-gu, Seoul, 명동, 서울, 대...   None

In [6]:
urls = df_4star['hotel_url'].tolist()
len(urls)

109

In [81]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [89]:
reviews = []

a = 105
b = 105
l = len(urls)

for i in range(a,l):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(5)
    
    # 언어 변경
    try:
        language_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select'))
        # driver.find_element(By.XPATH, '//*[@id="reviewFilterSection"]/div[1]/div[3]/span').click() #21번 전까지
        time.sleep(1)
    except ElementClickInterceptedException:
        continue

    # "한국어"를 선택
    try:
        language_dropdown.select_by_visible_text("한국어")
        # driver.find_element(By.XPATH, '//*[@id="reviews-language-filter_list"]/ul/li[3]').click() # 21번 전까지
        time.sleep(1)
    except ElementClickInterceptedException:
        continue
    
    while True:
        star_elements = driver.find_elements(By.CLASS_NAME, 'Review-comment-leftScore')
        review_elements = driver.find_elements(By.CLASS_NAME, 'Review-comment-bodyText')

        for star_element, review_element in zip(star_elements, review_elements):
            score = star_element.text
            review = review_element.text
            reviews.append([score, review])

        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
            time.sleep(1)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break
        except NoSuchElementException:
            break
        
    df_4star.at[i,'review'] = reviews
    reviews = []
    
    time.sleep(120)

In [ ]:
df_4star.tail(10)

In [ ]:
for i in range(len(urls)):
    review_column = df_4star.iloc[i]['review']
    
    if review_column is not None:
        print(f'{i}번째 : ', len(review_column))
    else:
        print(f'{i}번째 : None.')

In [93]:
df_4star.to_csv('./data/hotel_4star_agoda_add.csv',index=False,encoding='utf-8')

=======================

3성급

In [103]:
df_3star = pd.read_csv('./data/hotel_3star_agoda.csv')
df_3star.head()

Unnamed: 0                               hotel_name hotel_star  hotel_rate  \
0           0               호텔 파크하비오 (Hotel ParkHabio)        3성급         8.3   
1           1          호텔 페이토 강남 (Hotel Peyto Gangnam)        3성급         8.6   
2           2      도미인 서울 강남 (Dormy Inn SEOUL Gangnam)        3성급         9.1   
3           3  스타즈 호텔 명동 2호점 (Staz Hotel Myeongdong 2)        3성급         7.8   
4           4        롯데시티호텔 구로 (LOTTE City Hotel Guro)        3성급         8.8   

                                           hotel_url  \
0  https://www.agoda.com/ko-kr/hotel-parkhabio/ho...   
1  https://www.agoda.com/ko-kr/hotel-peyto-gangna...   
2  https://www.agoda.com/ko-kr/dormy-inn-seoul-ga...   
3  https://www.agoda.com/ko-kr/staz-hotel-myeongd...   
4  https://www.agoda.com/ko-kr/lotte-city-hotel-g...   

                                      hotel_location  \
0                      송파대로 111, 송파, 서울, 대한민국, 05837   
1                서초구 강남대로 259, 강남, 서울, 대한민국, 137-863   
2  134, Bongeunsa-ro, Gangnam-gu, 강남, 서울, 대한민국, 1...   
3  서울시 중구 수표로 16길 스타즈 호텔 명동 2, 명동, 서울, 대한민국, 100-013   
4                    디지털로 300, 구로, 서울, 대한민국, 152-848   

                                              review  
0  [['10.0', '문앞 바로 침대. 창문쪽으로 샤워실과 건조식 화장실 및 세면대....  
1  [['9.6', '당신은 지불한 것 이상을 얻습니다. 직원들은 매우 수용적이며 대부...  
2  [['10.0', '만약 알지 못하셨다면, 도미인은 일본의 호텔 체인입니다. 전반적...  
3  [['10.0', '이 호텔은 정말 가격 대비 가치가 있습니다. 👌🏻 명동 쇼핑 지...  
4  [['10.0', '저는 이 호텔을 여러 가지 이유로 정말 좋아합니다: 1. 위치와...

In [113]:
df_3star[50:70]

Unnamed: 0                                         hotel_name hotel_star  \
50          50                                   호텔 WD (Hotel WD)        3성급   
51          51  역삼 아르누보씨티 호텔 앤 레지던스 (Yeoksam Artnouveau City H...        3성급   
52          52                   호텔 안테룸 서울 (Hotel Anteroom Seoul)      3.5성급   
53          53                    통통 쁘띠호텔 (tong tong petit hotel)        3성급   
54          54                  도봉 호텔베이204 (DOBONG HOTEL BAY 204)        3성급   
55          55                                           HOTEL DM        3성급   
56          56                        더 가든 호텔 (The Garden Hotel )        3성급   
57          57                           르 서울 호텔 (LE SEOUL HOTEL)        3성급   
58          58                     가락 관광 호텔 (Karak Tourist Hotel)        3성급   
59          59              UH 스위트 더 명동 (UH Suite The Myeongdong)        3성급   
60          60  프렌들리 DH 네상스 호텔 바이 민드럼 그룹 (Friendly DH Naissanc...        3성급   
61          61                             호텔 아트리움 (Hotel Atrium)        NaN   
62          62      H 에비뉴 호텔 이대 신촌점 (H Avenue Hotel Idae Sinchon)        NaN   
63          63                      Toyoko Inn Seoul Dongdaemun I        NaN   
64          64        토요코인 서울 영등포 (Toyoko Inn Seoul Yeongdeungpo)        NaN   
65          65              호텔 미드시티 명동 (Hotel Midcity Myeongdong)        NaN   
66          66                                   Gold River Hotel        NaN   
67          67                                 주니비노 호텔 (junibino)        NaN   
68          68                  인사동 크라운 호텔 (Insadong Crown Hotel)        NaN   
69          69                        렉스 관광호텔 (LEX Tourist Hotel)        NaN   

    hotel_rate                                          hotel_url  \
50         8.7  https://www.agoda.com/ko-kr/hotel-wd/hotel/seo...   
51         7.2  https://www.agoda.com/ko-kr/yeoksam-artnouveau...   
52         8.5  https://www.agoda.com/ko-kr/hotel-anteroom-seo...   
53         9.3  https://www.agoda.com/ko-kr/tong-tong-petit-ho...   
54         8.2  https://www.agoda.com/ko-kr/dobong-hotel-bay-2...   
55         8.5  https://www.agoda.com/ko-kr/hotel-dm/hotel/seo...   
56         8.1  https://www.agoda.com/ko-kr/the-garedn-hotel/h...   
57         8.5  https://www.agoda.com/ko-kr/le-seoul-hotel/hot...   
58         7.5  https://www.agoda.com/ko-kr/karak-tourist-hote...   
59         8.3  https://www.agoda.com/ko-kr/uh-suite-the-myeon...   
60         8.6  https://www.agoda.com/ko-kr/dh-naissance-hotel...   
61         NaN  https://www.agoda.com/ko-kr/hotel-atrium/hotel...   
62         NaN  https://www.agoda.com/ko-kr/h-avenue-hotel-ida...   
63         NaN  https://www.agoda.com/ko-kr/toyoko-inn-seoul-d...   
64         NaN  https://www.agoda.com/ko-kr/toyoko-inn-seoul-y...   
65         NaN  https://www.agoda.com/ko-kr/afirst-hotel-myeon...   
66         NaN  https://www.agoda.com/ko-kr/gold-river-hotel-s...   
67         NaN  https://www.agoda.com/ko-kr/junibino-hotel/hot...   
68         NaN  https://www.agoda.com/ko-kr/insadong-crown-hot...   
69         NaN  https://www.agoda.com/ko-kr/lex-tourist-hotel/...   

                                       hotel_location review  
50                    관악구 신림로349, 관악, 서울, 대한민국, 08760   None  
51  701-1, Yeoksam-dong, Gangnam-gu, Seoul, 강남, 서울...   None  
52              153, Dosan-daero, 강남, 서울, 대한민국, 06037   None  
53                     율곡로10길 91, 종로, 서울, 대한민국, 03134   None  
54          서울 도봉구 도봉로 710 (방학동), 도봉, 서울, 대한민국, 01333   None  
55  217-19 Sungin-dong, Jongno-gu, 동대문, 서울, 대한민국, ...   None  
56  돈화문로 11나길 63 (63, Donhwamun-ro 11na-gil) The G...   None  
57    서울특별시 중구 남대문로 1길 56, 르서울호텔, 명동, 서울, 대한민국, 04526   None  
58                     송파대로28길 5, 송파, 서울, 대한민국, 05719   None  
59  49, Myeong-dong 8ga-gil, Jung-gu, Seoul, 명동, 서...   None  
60               성북구 동소문로 20나길 39, 강북, 서울, 대한민국, 2845   None  
61                                                NaN   None  
62                                                NaN 

In [ ]:
df_3star['review'] = None
df_3star.head()

In [115]:
urls = df_3star['hotel_url'].tolist()
len(urls)

148

In [183]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [164]:
reviews = []

a = 113
b = 60
l = len(urls)

for i in range(l-1,l):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(5)
    
    
    # 언어 변경
    try:
        language_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select'))
        # driver.find_element(By.XPATH, '//*[@id="reviewFilterSection"]/div[1]/div[3]/span').click() #21번 전까지
        time.sleep(1)
    except ElementClickInterceptedException:
        continue

    # "한국어"를 선택
    try:
        language_dropdown.select_by_visible_text("한국어")
        # driver.find_element(By.XPATH, '//*[@id="reviews-language-filter_list"]/ul/li[3]').click() # 21번 전까지
        time.sleep(1)
    except ElementClickInterceptedException:
        continue
    
    while True:
        star_elements = driver.find_elements(By.CLASS_NAME, 'Review-comment-leftScore')
        review_elements = driver.find_elements(By.CLASS_NAME, 'Review-comment-bodyText')

        for star_element, review_element in zip(star_elements, review_elements):
            score = star_element.text
            review = review_element.text
            reviews.append([score, review])

        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
            time.sleep(1)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break
        except NoSuchElementException:
            break
        
    df_3star.at[i,'review'] = reviews
    reviews = []
    
    time.sleep(120)

In [184]:
for i in range(61,len(urls)):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(2)
    
    # 홈페이지 총점 가져오기
    stars = float(driver.find_element('xpath','//*[@id="property-critical-root"]/div/div[5]/div[2]/div[1]/div[1]/div/div[1]/div/div/div/div/div/div/p[1]').text)
    df_3star.at[i,'hotel_rate'] = stars
        
    # 주소
    address = driver.find_element('xpath','//*[@id="property-main-content"]/div[1]/div[2]/div[2]/span[1]').text
    df_3star.at[i,'hotel_location'] = address
    
    
    # 성급
    star = driver.find_elements(By.CSS_SELECTOR,
                                    "#property-main-content > div.Box-sc-kv6pi1-0.cJiLOx.sc-gGmIRh.iyCRmc > div.HeaderCerebrum > div:nth-child(1) > div > span")
    if star:
        df_3star.at[i, "hotel_star"] = star[0].get_attribute('aria-label')
    else:
        df_3star.at[i, "hotel_star"] = None


In [185]:
df_3star[65:]

Unnamed: 0                                         hotel_name hotel_star  \
60           60  프렌들리 DH 네상스 호텔 바이 민드럼 그룹 (Friendly DH Naissanc...        3성급   
61           61                             호텔 아트리움 (Hotel Atrium)        3성급   
62           62      H 에비뉴 호텔 이대 신촌점 (H Avenue Hotel Idae Sinchon)        3성급   
63           63                      Toyoko Inn Seoul Dongdaemun I        3성급   
64           64        토요코인 서울 영등포 (Toyoko Inn Seoul Yeongdeungpo)        3성급   
..          ...                                                ...        ...   
143         143                               새턴 호텔 (SATURN HOTEL)        3성급   
144         144                         호텔 S 신촌 (Hotel S Shinchon)        3성급   
145         145                                  타오 호텔 (Hotel TAO)        3성급   
146         146                              호텔 윈저 (Hotel Windsor)        3성급   
147         147               에이퍼스트호텔 명동 (aFIRST Hotel Myeongdong)        3성급   

     hotel_rate                                          hotel_url  \
60          8.6  https://www.agoda.com/ko-kr/dh-naissance-hotel...   
61          8.2  https://www.agoda.com/ko-kr/hotel-atrium/hotel...   
62          8.0  https://www.agoda.com/ko-kr/h-avenue-hotel-ida...   
63          8.2  https://www.agoda.com/ko-kr/toyoko-inn-seoul-d...   
64          8.6  https://www.agoda.com/ko-kr/toyoko-inn-seoul-y...   
..          ...                                                ...   
143         8.6  https://www.agoda.com/ko-kr/saturn-hotel/hotel...   
144         8.3  https://www.agoda.com/ko-kr/hotel-s-shinchon/h...   
145         6.6  https://www.agoda.com/ko-kr/hotel-tao/hotel/se...   
146         9.5  https://www.agoda.com/ko-kr/hotel-windsor/hote...   
147         8.6  https://www.agoda.com/ko-kr/afirst-hotel-myeon...   

                                        hotel_location  \
60                성북구 동소문로 20나길 39, 강북, 서울, 대한민국, 2845   
61   106, Changgyeonggung-ro, Jongno-gu,Jongno-Gu,0...   
62                   신촌로 152, 마포구, 신촌, 서울, 대한민국, 04104   
63       337, Toegye-ro, Jung-gu, 동대문, 서울, 대한민국, 04565   
64                       신길로 293, 영등포, 서울, 대한민국, 07306   
..                                                 ...   
143                   자동차시장1길 29, 동대문, 서울, 대한민국, 04808   
144                     마포구 노고산동 109-154, 신촌, 서울, 대한민국   
145  863-24, Tongil-ro, Eunpyeong-gu, Seoul, 은평구, 서...   
146            독산동 1008-10, 금천구, 구로, 서울, 대한민국, 153-011   
147                        다동 130, 명동, 서울, 대한민국, 04522   

                                                review  
60   [[10.0, 시설이랑 청소상태 좋았고요, 위치도 성신여대 번화가 쪽이라 가족들이랑...  
61   [[10.0, 근처에 주차가능 한 곳이 별로 없더라구요 선착순 주차긴했지만 일찍가면...  
62   [[10.0, 세브란스 진료 받기위해 지방에서 올라와 하루 묵었는데 병원가기도 적당...  
63                                                None  
64   [[10.0, 서울에서 다른 숙소들에 실망감이 커서 여기도 별 기대없었는데 건물도 ...  
..                                                 ...  
143            [[10.0, 트윈베드 있는 방으로 예약했는데 무난하게 좋았어요 !]]  
144  [[8.0, 최고네요 ㅎㅎㅎ], [6.4, 어두운 분위기라 아이랑 같이 가기엔 비추...  
145  [[4.0, 해외있을때 예약했는데 투숙당일 연락와서는 싸게 팔고나서 싸게 팔았다고 ...  
146  [[2.0, 너무 지저분ᆢ침대 바닥에 쓴 콘돔이 돌아다니고 있었음], [4.4, 호...  
147  [[10.0, 설 연휴에 가족여행으로 갔는데 쾌적하고 너무 좋았습니다 명동, 광화문...  

[88 rows x 7 columns]

In [ ]:
for i in range(len(urls)):
    review_column = df_3star.iloc[i]['review']
    
    if review_column is not None:
        print(f'{i}번째 : ', len(review_column))
    else:
        print(f'{i}번째 : None.')

In [186]:
df_3star.to_csv('./data/hotel_3star_agoda_add.csv',index=False,encoding='utf-8')

===========================================================

지도 그리기

In [2]:
five_hotel = pd.read_csv('./data/hotel_5star_agoda_add.csv')
four_hotel = pd.read_csv('./data/hotel_4star_agoda_add.csv')
three_hotel = pd.read_csv('./data/hotel_3star_agoda_add.csv')

five_hotel = five_hotel[['hotel_name','hotel_location']]
four_hotel = four_hotel[['hotel_name','hotel_location']]
three_hotel = three_hotel[['hotel_name','hotel_location']]

In [3]:
def remove_parentheses(hotel_name):
    return re.sub(r'\([^)]*\)', '', hotel_name)

# 'Hotel_Name' 컬럼에 함수 적용
five_hotel['hotel_name'] = five_hotel['hotel_name'].apply(remove_parentheses)
four_hotel['hotel_name'] = four_hotel['hotel_name'].apply(remove_parentheses)
three_hotel['hotel_name'] = three_hotel['hotel_name'].apply(remove_parentheses)

In [4]:
five_hotel_address = five_hotel['hotel_location'].tolist()
four_hotel_address = five_hotel['hotel_location'].tolist()
three_hotel_address = five_hotel['hotel_location'].tolist()


In [5]:
five_hotel.tail()

hotel_name                                     hotel_location
34  JW 메리어트 동대문 스퀘어 서울                     청계천로 279 , 동대문, 서울, 대한민국, 03198
35  더 플라자 서울 오토그래프 컬렉션                 중구 소공로 119, 명동, 서울, 대한민국, (우) 04525
36     반얀트리 클럽 앤 스파 서울                  중구 장충동 2가 산5-5, 명동, 서울, 대한민국, 4605
37           3S 호텔 독산점         독산 3S부티크호텔, 시흥대로 353, 구로, 서울, 대한민국, 153-010
38         르메르디앙 서울 명동   38 Myeongdong 8na-gil, Jung-gu, 명동, 서울, 대한민국, ...

In [6]:
len(five_hotel['hotel_location'])

39

In [9]:
actions = ActionChains(driver)

In [14]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [15]:
urlurl = 'https://www.naver.com'
driver.get(urlurl)

In [27]:
five_hotel

hotel_name                 hotel_location
0                          그랜드 워커힐 서울              서울특별시 광진구 워커힐로 177
1                       스위스 그랜드 호텔 서울                 서울 광진구 워커힐로 177
2                          메이필드 호텔 서울                 서울 서대문구 연희로 353
3                      워커힐 더글라스 하우스  )                 서울 강서구 방화대로 94
4                             롯데호텔 월드                 서울 광진구 워커힐로 177
5                         이태원에 있는 숙소                                 
6                          비스타 워커힐 서울   서울 광진구 워커힐로 177 비스타 워커힐 서울 1층
7                          그랜드 하얏트 서울                 서울 광진구 워커힐로 177
8                     노보텔 앰배서더 서울 강남                   서울 용산구 소월로 322
9                              서울신라호텔                 서울 강남구 봉은사로 130
10                            롯데호텔 서울                   서울 중구 동호로 249
11                 노보텔 스위트 앰배서더 서울 용산                    서울 중구 을지로 30
12         소피텔 앰배서더 서울 호텔 & 서비스드 레지던스                서울 용산구 청파로20길 95
13                         어바웃스테이 신사점                  서울 송파구 잠실로 209
14                        몬드리안 서울 이태원                 서울 강남구 강남대로 620
15                      앰배서더 서울 풀만 호텔                   서울 용산구 장문로 23
16                 호텔 나루 서울 엠갤러리 앰배서더                   서울 중구 동호로 287
17                            시그니엘 서울              서울특별시 송파구 올림픽로 300
18                      인터컨티넨탈 서울 코엑스                 서울 송파구 올림픽로 300
19                           파라스파라 서울                 서울 강남구 봉은사로 524
20                       페어몬트 앰배서더 서울              서울시 영등포구 여의대로 108,
21                             콘래드 서울                서울 영등포구 여의대로 108
22           조선 팰리스 서울 강남 럭셔리 컬렉션 호텔                 서울 영등포구 국제금융로 10
23                         밀레니엄 힐튼 서울                 서울 강남구 테헤란로 231
24                 그랜드 인터컨티넨탈 서울 파르나스                    서울 중구 소월로 50
25                      JW 메리어트 호텔 서울                 서울 강남구 테헤란로 521
26       그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산                 서울 서초구 신반포로 176
27                          안다즈 서울 강남                서울 용산구 청파로20길 95
28                          파크 하얏트 서울                  서울 강남구 논현로 854
29                         포시즌스 호텔 서울                 서울 강남구 테헤란로 606
30                   롯데호텔서울 이그제큐티브 타워                  서울 종로구 새문안로 97
31  여의도 파크 센터, 서울 - 메리어트 이그제큐티브 아파트먼트                    서울 중구 을지로 30
32                          웨스틴 조선 서울                  서울 영등포구 여의대로 8
33             Seoul D Cube City Hotel                  서울 중구 소공로 106
34                 JW 메리어트 동대문 스퀘어 서울              서울특별시 종로구 청계천로 279
35                 더 플라자 서울 오토그래프 컬렉션                 서울 종로구 청계천로 279
36                    반얀트리 클럽 앤 스파 서울                   서울 중구 소공로 119
37                          3S 호텔 독산점                   서울 중구 장충단로 60
38                        르메르디앙 서울 명동                 서울 금천구 시흥대로 353

In [21]:
a = 0
l = len(five_hotel['hotel_name'])

for i in range(a,l):
    try:
        driver.find_element('xpath','//*[@id="nx_search_form"]/fieldset/button/i').click()
        time.sleep(1)
        driver.find_element('xpath','//*[@id="nx_query"]').clear()
        driver.find_element('xpath','//*[@id="nx_query"]').send_keys(five_hotel.iloc[i]['hotel_name'])
        time.sleep(1)
        new = driver.find_element('xpath','//*[@id="place-main-section-root"]/section[1]/div/div[3]/div/div/div[1]/div/a/span[1]').text
        five_hotel.at[i,'hotel_location'] = new
        time.sleep(1)
    except:
        continue
    

In [26]:
five_hotel.at[0,'hotel_location'] = '서울특별시 광진구 워커힐로 177'
five_hotel.at[6,'hotel_location'] = '서울 광진구 워커힐로 177 비스타 워커힐 서울 1층'
five_hotel.at[17,'hotel_location'] = '서울특별시 송파구 올림픽로 300'
five_hotel.at[34,'hotel_location'] = '서울특별시 종로구 청계천로 279'

five_hotel.at[5,'hotel_location'] = ''

In [28]:
for i in range(len(five_hotel['hotel_location'])):
    location = five_hotel.iloc[i]['hotel_location']
    
    if isinstance(location, str):
        a = location.split(' ')
        five_hotel.at[i, 'hotel_location'] = " ".join(a[0:4])
    else:
        print(f"{i}번째 : {location}")


In [ ]:
five_hotel

In [ ]:
# 4성급
four_hotel.head(10)

In [32]:
a = 0
l = len(four_hotel['hotel_name'])

for i in range(a,l):
    try:
        driver.find_element('xpath','//*[@id="nx_query"]').clear()
        driver.find_element('xpath','//*[@id="nx_query"]').send_keys(four_hotel.iloc[i]['hotel_name'])
        time.sleep(1)
        driver.find_element('xpath','//*[@id="nx_search_form"]/fieldset/button/i').click()
        time.sleep(1)
        new = driver.find_element('xpath','//*[@id="place-main-section-root"]/section[1]/div/div[3]/div/div/div[1]/div/a/span[1]').text
        four_hotel.at[i,'hotel_location'] = new
        time.sleep(1)
    except:
        continue
    

In [39]:
four_hotel.at[23,'hotel_location'] = '서울특별시 동작구 상도로 120'
four_hotel.at[58,'hotel_location'] = '서울특별시 중구 장충단로13길 20'
four_hotel.at[68,'hotel_location'] = '서울특별시 구로구 신도림동 432-30'
four_hotel.at[75,'hotel_location'] = '서울특별시 영등포구 양평로 132'
four_hotel.at[82,'hotel_location'] = '서울특별시 중구 동호로 335'
four_hotel.at[86,'hotel_location'] = '서울특별시 중구 명동9길 16'
four_hotel.at[88,'hotel_location'] = '서울특별시 강남구 도산대로 203'
four_hotel.at[90,'hotel_location'] = '서울특별시 중구 명동7가길 22'

In [41]:
for i in range(len(four_hotel['hotel_location'])):
    location = four_hotel.iloc[i]['hotel_location']
    
    if isinstance(location, str):
        a = location.split(' ')
        four_hotel.at[i, 'hotel_location'] = " ".join(a[0:4])
    else:
        print(f"{i}번째 : {location}")


6번째 : nan


In [ ]:
three_hotel[100:]

In [45]:
a = 0
l = len(three_hotel['hotel_name'])

for i in range(a,l):
    try:
        driver.find_element('xpath','//*[@id="nx_query"]').clear()
        driver.find_element('xpath','//*[@id="nx_query"]').send_keys(three_hotel.iloc[i]['hotel_name'])
        time.sleep(1)
        driver.find_element('xpath','//*[@id="nx_search_form"]/fieldset/button/i').click()
        time.sleep(1)
        new = driver.find_element('xpath','//*[@id="place-main-section-root"]/section[1]/div/div[3]/div/div/div[1]/div/a/span[1]').text
        three_hotel.at[i,'hotel_location'] = new
        time.sleep(3)
    except:
        continue

In [387]:
new = driver.find_element('xpath','//*[@id="loc-main-section-root"]/section/div/div[3]/div[1]/div[1]/span').text
three_hotel.at[147,'hotel_location'] = new

In [57]:
three_hotel.at[44,'hotel_location'] = '서울 관악구 남부순환로 1458'
three_hotel.at[51,'hotel_location'] = '서울특별시 강남구 역삼동 701-1'
three_hotel.at[59,'hotel_location'] = '서울특별시 중구 명동8가길 49'
three_hotel.at[60,'hotel_location'] = '서울특별시 성북구 동소문로20나길 39'
three_hotel.at[61,'hotel_location'] = '서울 종로구 창경궁로 106'
three_hotel.at[63,'hotel_location'] = '서울특별시 중구 퇴계로 337'
three_hotel.at[68,'hotel_location'] = '서울 종로구 삼일대로30길 9'
three_hotel.at[79,'hotel_location'] = '서울특별시 성동구 도선동 72'
three_hotel.at[80,'hotel_location'] = '서울특별시 서초구 나루터로 64'
three_hotel.at[84,'hotel_location'] = '서울특별시 영등포구 영등포동3가 12-22'
three_hotel.at[85,'hotel_location'] = '서울특별시 송파구 올림픽로32길 7'
three_hotel.at[88,'hotel_location'] = '서울특별시 중구 만리동1가 33-1'
three_hotel.at[89,'hotel_location'] = '서울특별시 중구 장충단로 198'
three_hotel.at[91,'hotel_location'] = '서울특별시 동대문구 천호대로 377'
three_hotel.at[95,'hotel_location'] = '서울특별시 중구 명동9길 22'
three_hotel.at[108,'hotel_location'] = '서울특별시 중구 퇴계로 107'
three_hotel.at[113,'hotel_location'] = '서울 종로구 청계천로 137'
three_hotel.at[118,'hotel_location'] = '서울특별시 중구 퇴계로 139'
three_hotel.at[126,'hotel_location'] = '서울특별시 중구 퇴계로12길 39-8'
three_hotel.at[129,'hotel_location'] = '서울특별시 중구 명동8나길 38'
three_hotel.at[132,'hotel_location'] = '서울특별시 중구 세종대로22길 12'
three_hotel.at[133,'hotel_location'] = '서울 종로구 돈화문로9길 29'
three_hotel.at[134,'hotel_location'] = '서울특별시 노원구 상계6.7동 702-1'
three_hotel.at[146,'hotel_location'] = '서울특별시 금천구 독산동 1008-10'
three_hotel.at[147,'hotel_location'] = '서울특별시 중구 다동 130'

In [58]:
for i in range(len(three_hotel['hotel_location'])):
    location = three_hotel.iloc[i]['hotel_location']
    
    if isinstance(location, str):
        a = location.split(' ')
        three_hotel.at[i, 'hotel_location'] = " ".join(a[0:4])
    else:
        print(f"{i}번째 : {location}")

5번째 : nan


In [ ]:
!pip install geopy

In [56]:
####### 도로명주소 위도 경도 값으로 바꿔주기 ########
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [ ]:
latitude = []
longitude =[]

for i in address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [59]:
five_hotel.to_csv('./data/five_hotel.csv',index=False,encoding='utf-8')
four_hotel.to_csv('./data/four_hotel.csv',index=False,encoding='utf-8')
three_hotel.to_csv('./data/three_hotel.csv',index=False,encoding='utf-8')